<a href="https://colab.research.google.com/github/ucalyptus/BS-Nets-Implementation-Pytorch/blob/master/DA_RecNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization, ReLU, PReLU, MaxPool3D, Conv3DTranspose
from keras.layers import Dropout, Input, GlobalAveragePooling2D, multiply, add, Activation, Permute
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras import regularizers
from keras import backend as K
import tensorflow as tf
from keras.layers import Layer

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from operator import truediv

# from plotly.offline import init_notebook_mode
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
# import spectral

# init_notebook_mode(connected=True)
%matplotlib inline

Using TensorFlow backend.


In [0]:
!pip install -U spectral
if not (os.path.isfile('/content/Indian_pines_corrected.mat')):
  !wget http://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat
if not (os.path.isfile('/content/Indian_pines_gt.mat')):
  !wget http://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat

     |████████████████████████████████| 153kB 6.6MB/s 
  Created wheel for spectral: filename=spectral-0.20-cp36-none-any.whl size=183918 sha256=f807d5f8c115f87d90e15fb6f388e34c8357ff45280d89fdafc38a2284784f4c
  Stored in directory: /root/.cache/pip/wheels/ad/cf/f3/3cab28f6aed46f15c8db09c6ad678483610426261025e61ff8
Successfully built spectral
--2019-11-01 20:07:50--  http://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat
Resolving www.ehu.eus (www.ehu.eus)... 158.227.0.65, 2001:720:1410::65
Connecting to www.ehu.eus (www.ehu.eus)|158.227.0.65|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5953527 (5.7M) [text/plain]
Saving to: ‘Indian_pines_corrected.mat’

Indian_pines_correc 100%[===================>]   5.68M  1.53MB/s    in 3.7s    

2019-11-01 20:07:54 (1.53 MB/s) - ‘Indian_pines_corrected.mat’ saved [5953527/5953527]

--2019-11-01 20:07:59--  http://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat
Resolving www.ehu.eus (www.ehu.eus)... 15

In [0]:
def loadData():
    data = sio.loadmat('Indian_pines_corrected.mat')['indian_pines_corrected']
    labels = sio.loadmat('Indian_pines_gt.mat')['indian_pines_gt']
    
    return data, labels

In [0]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [0]:
def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [0]:
X, y = loadData()

In [0]:
X, y = createImageCubes(X, y)

X.shape, y.shape

((10249, 5, 5, 200), (10249,))

# Model and Training

In [0]:
# X = X.reshape(-1, 200, 5, 5)
# X.shape

In [0]:
# def BAM():
#     model = Sequential()
#     model.add(Conv2D(filters=64,
#                      input_shape=(200, 5, 5),
#                      kernel_size=(3,3),
#                      strides=1,
#                      padding='valid', name="Conv1"))
#     model.add(ReLU(name="ReLU1"))
#     model.add(GlobalAveragePooling2D(data_format="channels_first"))
    
#     model.add(Dense(128))
#     model.add(ReLU(name="ReLU2"))
#     model.add(Dense(200, activation="sigmoid"))
  
#     return model

In [0]:
class PAM(Layer):
    def __init__(self,
                 gamma_initializer=tf.zeros_initializer(),
                 gamma_regularizer=None,
                 gamma_constraint=None,
                 **kwargs):
        super(PAM, self).__init__(**kwargs)
        self.gamma_initializer = gamma_initializer
        self.gamma_regularizer = gamma_regularizer
        self.gamma_constraint = gamma_constraint

    def build(self, input_shape):
        self.gamma = self.add_weight(shape=(1, ),
                                     initializer=self.gamma_initializer,
                                     name='gamma',
                                     regularizer=self.gamma_regularizer,
                                     constraint=self.gamma_constraint)

        self.built = True

    def compute_output_shape(self, input_shape):
        return input_shape

    def call(self, input):
        input_shape = input.get_shape().as_list()
        _, h, w, filters = input_shape

        b = Conv2D(filters // 8, 1, use_bias=False, kernel_initializer='he_normal')(input)
        c = Conv2D(filters // 8, 1, use_bias=False, kernel_initializer='he_normal')(input)
        d = Conv2D(filters, 1, use_bias=False, kernel_initializer='he_normal')(input)

        vec_b = K.reshape(b, (-1, h * w, filters // 8))
        vec_cT = tf.transpose(K.reshape(c, (-1, h * w, filters // 8)), (0, 2, 1))
        bcT = K.batch_dot(vec_b, vec_cT)
        softmax_bcT = Activation('softmax')(bcT)
        vec_d = K.reshape(d, (-1, h * w, filters))
        bcTd = K.batch_dot(softmax_bcT, vec_d)
        bcTd = K.reshape(bcTd, (-1, h, w, filters))

        out = self.gamma*bcTd + input
        return out


class CAM(Layer):
    def __init__(self,
                 gamma_initializer=tf.zeros_initializer(),
                 gamma_regularizer=None,
                 gamma_constraint=None,
                 **kwargs):
        super(CAM, self).__init__(**kwargs)
        self.gamma_initializer = gamma_initializer
        self.gamma_regularizer = gamma_regularizer
        self.gamma_constraint = gamma_constraint

    def build(self, input_shape):
        self.gamma = self.add_weight(shape=(1, ),
                                     initializer=self.gamma_initializer,
                                     name='gamma',
                                     regularizer=self.gamma_regularizer,
                                     constraint=self.gamma_constraint)

        self.built = True

    def compute_output_shape(self, input_shape):
        return input_shape

    def call(self, input):
        input_shape = input.get_shape().as_list()
        _, h, w, filters = input_shape

        vec_a = K.reshape(input, (-1, h * w, filters))
        vec_aT = tf.transpose(vec_a, (0, 2, 1))
        aTa = K.batch_dot(vec_aT, vec_a)
        softmax_aTa = Activation('softmax')(aTa)
        aaTa = K.batch_dot(vec_a, softmax_aTa)
        aaTa = K.reshape(aaTa, (-1, h, w, filters))

        out = self.gamma*aaTa + input
        return out
      
def DA():
    input_layer = Input((5, 5, 200))
    pam = PAM()(input_layer)
    cam = CAM()(input_layer)
    
    feature_sum = add([pam, cam])
    
    return Model(inputs=input_layer, outputs=feature_sum)

In [0]:
def DCAE(weight_decay=0.0005):
    model = Sequential()
    model.add(Conv3D(filters=24,
                     input_shape=(200, 5, 5, 1),
                     kernel_size=(24, 3, 3),
                     strides=(1, 1, 1),
                     kernel_regularizer=regularizers.l2(l=weight_decay),
                     padding='valid', name="Conv1"))
    model.add(BatchNormalization(name="BN1"))
    model.add(PReLU(name="PReLU1"))

    model.add(Conv3D(filters=48,
                     kernel_size=(24, 3, 3),
                     strides=(1, 1, 1),
                     kernel_regularizer=regularizers.l2(l=weight_decay),
                     padding='valid', name="Conv2"))
    model.add(BatchNormalization(name="BN2"))
    model.add(PReLU(name="PReLU2"))

    model.add(MaxPool3D(pool_size=(18, 1, 1),
                        strides=(18, 1, 1), name="Pool1"))

    model.add(Conv3DTranspose(filters=24,
                              kernel_size=(9, 3, 3),
                              kernel_regularizer=regularizers.l2(
                                  l=weight_decay),
                              strides=(22, 1, 1), name="Deconv1", padding='valid'))
    model.add(BatchNormalization(name="BN3"))
    model.add(PReLU(name="PReLU3"))
    model.add(Conv3DTranspose(filters=1,
                              kernel_size=(25, 3, 3),
                              kernel_regularizer=regularizers.l2(
                                  l=weight_decay),
                              strides=(1, 1, 1), name="Deconv2", padding='valid'))
    model.add(BatchNormalization(name="BN4"))
    
    return model

In [0]:
def Ensemble():
    input_layer = Input((5, 5, 200))
#     band_activations = BAM()(input_layer)
#     band_activations = Reshape((200, 1, 1))(band_activations)
    
#     bam_output = multiply([band_activations, input_layer])
    
#     bam_output = Reshape((200, 5, 5, 1))(bam_output)
    da_output = DA()(input_layer)
    da_output = Permute((3,1,2), input_shape=(5,5,200))(da_output)
    da_output = Reshape((200, 5, 5, 1))(da_output)
    output = DCAE()(da_output)
    
    
    return Model(inputs=input_layer, outputs=output)
    

In [0]:
# model = DCAE(weight_decay=0.0005)
model = Ensemble()
model.summary()






Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5, 5, 200)         0         
_________________________________________________________________
model_1 (Model)              (None, 5, 5, 200)         2         
_________________________________________________________________
permute_1 (Permute)          (None, 200, 5, 5)         0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 200, 5, 5, 1)      0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 200, 5, 5, 1)      436853    
Total params: 436,855
Trainable params: 436,661
Non-trainable params: 194
_________________________________________________________________


In [0]:
def psnr(x_true, x_pred):
    n_samples = x_true.shape[0]
    n_bands = x_true.shape[1]
    PSNR = np.zeros(n_bands)
    MSE = np.zeros(n_bands)
    mask = np.ones(n_bands)
    for k in range(n_bands):
        x_true_k = x_true[:, k].reshape([-1])
        x_pred_k = x_pred[:, k].reshape([-1])
        MSE[k] = 1.0 / n_samples * mean_squared_error(x_true_k, x_pred_k, )
        MAX_k = np.max(x_true_k)
        if MAX_k != 0:
            PSNR[k] = 10 * math.log10(math.pow(MAX_k, 2) / MSE[k])
        else:
            mask[k] = 0

    psnr = PSNR.sum()/mask.sum()
    mse = MSE.mean()
    print('psnr', psnr)
    print('mse', mse)
    
    return psnr, mse

In [0]:
import scipy
from scipy.special import kl_div
def Dskl(Bi,Bj):
  
  
  pk = np.histogramdd(np.ravel(Bi), bins = 256)[0]/Bi.size 
  #pk = list(filter(lambda p: p > 0, pk))
  pk = np.array(pk)

  qk = np.histogramdd(np.ravel(Bj), bins = 256)[0]/Bj.size
  #qk = list(filter(lambda p: p > 0, qk)
  qk = np.array(qk)
  
  
  S = kl_div(pk,qk) + kl_div(qk,pk)
  
  #S = np.sum(pk * np.log2(pk / qk), axis=0) + np.sum(qk * np.log2(qk / pk), axis=0)
  
  return S

In [0]:
ENTROPY = np.zeros(200)
import skimage    
def top15bands(x_predict):
  for i in range(0,len(ENTROPY)):
    ENTROPY[i]+=skimage.measure.shannon_entropy(x_predict[:,i,:,:])
  
  print('Top 15 bands with Entropy ->',ENTROPY.argsort()[-15:][::-1])

In [0]:
def MSD(x_predict):
  top15 = ENTROPY.argsort()[-15:][::-1]
  perbatch = list(0 for i in range(0,160))
  for batch_idx in range(0,len(perbatch)):
    for i in range(0,len(top15)):
      for j in range(0,i):
        perbatch[batch_idx]+=Dskl(x_predict[:,top15[i],:,:],x_predict[:,top15[j],:,:])
  perbatch = np.array(perbatch)
  perbatch[perbatch>=1E03]=0
  print('Mean of MSD with Top 15 bands is \n',np.mean(perbatch)*100)



In [0]:
from keras.callbacks import Callback
from sklearn.metrics import mean_squared_error
import math

class MyLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        x_predict = model.predict(X)
        x_true = np.asarray(X)
        x_pred_centre = x_predict[:, :, 2, 2]
        x_true_centre = x_true[:, :, 2, 2]
        psnr(x_true_centre, x_pred_centre)
        top15bands(x_predict)
    def on_train_end(self,logs=None):
      x_predict = model.predict(X)
      MSD(x_predict)

In [0]:
model.compile(loss=keras.losses.mse, optimizer=keras.optimizers.Adam(lr=0.1))

n_epoch = 100


model.fit(X, X.reshape(-1, 200, 5, 5, 1), epochs=n_epoch, shuffle=True, verbose=1, batch_size=32, callbacks=[MyLogger()])

Epoch 1/1
10249/10249 [==============================] - 16s 2ms/step - loss: 5070529.8298
psnr 45.73431634597225
mse 1041.1729192428718
Top 15 bands with Entropy -> [ 12   4  20   5  52  13 140  53  36  76  29  28 116 157  21]
Mean of MSD with Top 15 bands is 
 54.6476714974499
